In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings


In [2]:
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)

In [3]:
df = pd.read_csv("employee.csv")

In [4]:
df.shape

(4277, 13)

In [6]:
df.head()

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [7]:
df = df.drop(columns=['id', 'timestamp','country'])

In [8]:
df.shape

(4277, 10)

In [9]:
df.columns

Index(['employment_status', 'job_title', 'job_years', 'is_manager',
       'hours_per_week', 'telecommute_days_per_week', 'education',
       'is_education_computer_related', 'certifications', 'salary'],
      dtype='object')

In [10]:
# replace nan with median value for hours_per_week
df.loc[df['hours_per_week'].isna(), 'hours_per_week'] = df['hours_per_week'].median()


In [11]:
# replace nan with median value for telecommute_days_per_week
df.loc[df['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = df['telecommute_days_per_week'].median()

In [12]:
#drop other nans
df = df.dropna()

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB


## Feature Encoding

In [15]:
df_train = df.copy()

In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4261 entries, 0 to 4276
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   employment_status              4261 non-null   object 
 1   job_title                      4261 non-null   object 
 2   job_years                      4261 non-null   float64
 3   is_manager                     4261 non-null   object 
 4   hours_per_week                 4261 non-null   float64
 5   telecommute_days_per_week      4261 non-null   float64
 6   education                      4261 non-null   object 
 7   is_education_computer_related  4261 non-null   object 
 8   certifications                 4261 non-null   object 
 9   salary                         4261 non-null   float64
dtypes: float64(4), object(6)
memory usage: 366.2+ KB


In [17]:
df_train.head()

,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [18]:
#Encoding binary columns
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    df_train[c] = df_train[c].replace(to_replace=['Yes'], value=1)
    df_train[c] = df_train[c].replace(to_replace=['No'], value=0)

In [20]:
# Categorical Features
col_cat = [c for c in df_train.columns if df_train[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
df_cat = df_train[col_cat]

In [21]:
df_cat.head()

,employment_status,job_title,education,is_education_computer_related
0,Full time,Developer,Bachelors (4 years),Yes
1,Full time,DBA,Bachelors (4 years),Yes
2,Full time,DBA,Masters,Yes
3,Full time,DBA,Bachelors (4 years),No
4,Full time,Developer,Masters,Yes


In [22]:
fin = pd.get_dummies(df_train, columns=col_cat, drop_first= True,dtype=int)
fin.shape

(4261, 25)

In [23]:
fin.head()

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.78393,1,40.0,0.0,0,7187.743094,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
1,5.00000,0,40.0,5.0,0,10000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1.00000,0,40.0,0.0,1,7000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1.00000,0,40.0,2.0,0,8333.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,3.00000,0,40.0,2.0,1,7137.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1


In [24]:
fin

,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,salary,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
0,4.783930,1,40.0,0.0,0,7187.743094,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
1,5.000000,0,40.0,5.0,0,10000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,1.000000,0,40.0,0.0,1,7000.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,1.000000,0,40.0,2.0,0,8333.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,3.000000,0,40.0,2.0,1,7137.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,2.000000,1,40.0,0.0,0,4917.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4273,4.140793,1,40.0,0.0,0,7033.845701,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4274,5.000000,0,40.0,0.0,0,7166.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4275,4.000000,1,40.0,2.0,0,9583.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


## Test Train split


In [25]:
y = fin['salary']
X = fin.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)

Training Set Dimensions: (3408, 24)
Validation Set Dimensions: (853, 24)


In [26]:
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']
num_cols

['job_years', 'hours_per_week', 'telecommute_days_per_week']

In [27]:
#Standerd scaling
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [28]:
#Linear Regression
reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [29]:
reg.coef_

array([  185.01169844,    56.40957964,   217.27881576,   226.20961927,
         108.10455286,   -67.52201313,   243.03181828, -2242.37077369,
        1083.9743616 ,   848.0587822 ,   790.0602497 ,   552.59276212,
         995.83983101,   788.87714606,   123.01501008,   980.20979069,
         734.12888094,  3539.12439783,   334.55560311,    11.02825614,
         204.51370136,   236.003842  ,  -346.64589237,  -216.81814996])

In [30]:
reg.intercept_

np.float64(6159.8417805484005)

In [31]:
#MSE
mean_squared_error(y_train,reg.predict(X_train))/np.mean(y_train)

np.float64(213.27070606470852)

In [32]:
#Predicted values
y_pred = reg.predict(X_test)

In [34]:
#Test date evaluation
mse = mean_squared_error(y_pred, y_test)/np.mean(y_test)
print("Mean Squared Error:", mse)

Mean Squared Error: 14014.331193181657


In [35]:
scaler.fit(X_test[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [36]:
y_hat = reg.predict(X_test)

In [37]:
#Predicted Values
y_hat

array([7500.65071113, 7836.06660234, 6618.61963866, 7036.28891642,
       7414.65896351, 6529.69503578, 7104.51612092, 7350.51936197,
       5571.5041311 , 7014.96061177, 7057.30148145, 6995.44582463,
       6519.00161431, 6707.54000187, 7138.51665213, 7164.09994297,
       7511.84907583, 7574.4824017 , 7181.40030024, 5959.85063072,
       7246.5912557 , 6515.62632581, 7932.81937729, 7308.75600597,
       6938.70414326, 6849.53649344, 7660.12358768, 6732.95425743,
       6888.00162641, 8176.48228112, 7032.29377985, 7278.37684732,
       7121.00233554, 6748.44738106, 6986.11222065, 7779.09445529,
       6668.67803335, 7570.26679285, 6474.2718794 , 7880.41468868,
       7077.08543848, 7164.69517142, 6957.64104631, 6161.9735447 ,
       6657.70936791, 6542.45491001, 7241.88981651, 8040.89059173,
       7145.00251352, 6443.42856923, 7149.27883295, 6579.0614978 ,
       7512.4696569 , 6952.21141196, 6632.71834348, 6872.94471056,
       7272.96175227, 6986.11222065, 7080.48854157, 6928.18436

In [40]:
# Normalized Mean Absolute Error
test= mean_absolute_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Abosulte Error: ", test)

Test Mean Abosulte Error:  0.11445520364834745


In [41]:
# Normalized Mean Squared Error
mse = mean_squared_error(y_test,y_hat)/np.mean(y_hat)
print("Test Mean Squared Error: ",mse)

Test Mean Squared Error:  193.6721870562702
